In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from trackml.dataset import load_event, load_dataset 
from trackml.randomize import shuffle_hits
from trackml.score import score_event 
import os
import math
%run utils.ipynb

In [ ]:
#hits, cells, particles, truth = load_data_single_event(1000)

In [2]:
"""
    hits: tensor of samples x timesteps x features
    bounds: K:V of x_max, x_min, y_max, y_min, z_max, z_min
"""

"""
assuming input is samples x [x_val,y_val,z_val]
"""
def calc_norm(xyz, x_min, x_max, y_min, y_max, z_min, z_max):
    norm_x = ((xyz[0] - x_min)/(x_max - x_min))
    norm_y = ((xyz[1] - y_min)/(y_max - y_min))
    norm_z = ((xyz[2] - z_min)/(z_max - z_min))
    return [norm_x,norm_y,norm_z]

def norm_hits(hits, bounds):
    assert ('x_max' and 'x_min' and 'y_max' and 'y_min' and 'z_max' and 'z_min' in bounds) or ('r_max' and 'r_min' and 'phi_max' and 'phi_min' and 'z_max' and 'z_min' in bounds)
    return np.apply_along_axis(calc_norm, 1, hits, x_min = bounds['x_min'], x_max = bounds['x_max'], y_min = bounds['y_min'], y_max = bounds['y_max'], z_min = bounds['z_min'], z_max = bounds['z_max']) 


def calc_bounds():
    bounds={'x_max':0, 'x_min':0, 'y_max':0, 'y_min':0, 'z_max':0, 'z_min':0}
    for num in range(1000,1099):
        file_name = 'event00000' + str(num)
        hits, cells, particles, truth = load_event('data/train_sample/'+file_name)
        event_x_max, event_x_min, event_y_max, event_y_min, event_z_max, event_z_min = hits['x'].max(), hits['x'].min(), hits['y'].max(), hits['y'].min(), hits['z'].max(), hits['z'].min() 
        if bounds['x_max'] < event_x_max:
            bounds['x_max'] = event_x_max
        if bounds['x_min'] > event_x_min:
            bounds['x_min'] = event_x_min
        if bounds['y_max'] < event_y_max:
            bounds['y_max'] = event_y_max
        if bounds['y_min'] > event_y_min:
            bounds['y_min'] = event_y_min
        if bounds['z_max'] < event_z_max:
            bounds['z_max'] = event_z_max
        if bounds['z_min'] > event_y_min:
            bounds['z_min'] = event_y_min
    return bounds
        

"""
input: hits and truth dfs

output: df of particle_id as index and list of particles 
"""

def gen_tracks(truth_df):
    assert(isinstance(truth_df, pd.DataFrame))
    #print(truth_df)
    truth_df['dist'] = np.sqrt(truth_df['tx']**2+truth_df['ty']**2+truth_df['tz']**2)
    grouped = truth_df.groupby('particle_id')['hit_id','dist']
    a = grouped.apply(lambda x: x.sort_values(by=['dist'],ascending=True))
    final = a.groupby('particle_id')['hit_id'].apply(np.array)
    return final 

##there's still a problem with how to deal with hits that have a particle id of 0

def batch_iter(truth_df, batch_size):
    tracks = gen_tracks(truth_df).values
    np.random.shuffle(tracks) 
    remainder = len(tracks) % batch_size if len(tracks) % batch_size is not 0 else 0
    if remainder is not 0:
        modded_tracks = tracks[:-remainder]
    else:
        modded_tracks = tracks 
    assert(len(modded_tracks)%batch_size is 0)
    for batch in modded_tracks.reshape(-1,batch_size,1):
        yield batch
        
def get_data(max_seq_len, batch_size, feature_len, truth_df, hits_df):
    hits = hits_df
    max_seq_len = max_seq_len
    b_size = batch_size
    features = feature_len #xyz or phi r z
    all_data = list(batch_iter(truth_df,b_size))
    
    #print(all_data)
    for result in all_data:
        batch = []
        batch_lv = []
        labels_tensor = []
        labels_tensor_lv = []
        for track_list in result:
            for hit_id in track_list:
                hit_coord = []
                track = []
                track_lb = []
                lv_pair = []
                lv_tensor = []
                label_coord = []
                for elem in hit_id:
                    x, y, z, layer_id, volume_id = hits.loc[hits['hit_id']== elem]['x'].item(), hits.loc[hits['hit_id']== elem]['y'].item(), hits.loc[hits['hit_id']== elem]['z'].item(), hits.loc[hits['hit_id'] == elem]['volume_id'].item(), hits.loc[hits['hit_id'] == elem]['layer_id'].item()
                    r,phi,z = cartesian_to_3d_polar(x,y,z)
                    hit_coord = [r,phi,z,layer_id, volume_id]
                    label_coord = [r,phi,z]
                    track.append(hit_coord)
                    track_lb.append(label_coord)
                    layer, volume = hits.loc[hits['hit_id'] == elem]['volume_id'].item(), hits.loc[hits['hit_id'] == elem]['layer_id'].item()
                    lv_pair = [layer, volume]
                    lv_tensor.append(lv_pair)
                zeros_to_add = max_seq_len - len(track)
                if zeros_to_add > 0:
                    add_array = np.zeros((zeros_to_add,feature_len))
                    add_array_lb = np.zeros((zeros_to_add, 3)) #3 is hardcoded for xyz/rphiz
                    add_array_lv = np.zeros((zeros_to_add, 2))
                    np_data = np.array(track)
                    np_data_lb = np.array(track_lb)
                    np_data_lv = np.array(lv_tensor)
                    padded_track_data  = np.append(np_data,add_array,axis=0)
                
                    padded_track_data_lb = np.append(np_data_lb, add_array_lb, axis=0)
                    padded_track_data_lv = np.append(np_data_lv, add_array_lv, axis=0)
                elif zeros_to_add < 0:
                    modded_track = track[:zeros_to_add]
                    modded_track_lv = lv_tensor[:zeros_to_add]
                    modded_track_lb = track_lb[:zeros_to_add]
                    padded_track_data_lb = np.array(modded_track_lb)
                    padded_track_data = np.array(modded_track)
                    padded_track_data_lv = np.array(modded_track_lv)
                else:
                    padded_track_data_lb = np.array(track_lb)
                    padded_track_data = np.array(track)
                    padded_track_data_lv = np.array(lv_tensor)
            
            row_label = padded_track_data_lb[1:]
            row_label_lv = padded_track_data_lv[1:]
            padded_row_label = np.append(row_label, np.zeros((1,3)), axis=0) #hardcoded 3 for xyz/rphiz
            padded_row_label_lv = np.append(row_label_lv, np.zeros((1,2)), axis=0)
            labels_tensor.append(padded_row_label)
            labels_tensor_lv.append(padded_row_label_lv)
            batch.append(padded_track_data)
            batch_lv.append(padded_track_data_lv)
            
        padded_batch_data = np.array(batch)
        padded_batch_data_lv = np.array(batch_lv)
        padded_labels = np.array(labels_tensor)
        padded_labels_lv = np.array(labels_tensor_lv)
        #print(padded_labels)
        #print(padded_labels_lv)
        yield padded_batch_data, padded_labels, padded_batch_data_lv, padded_labels_lv
        
def next_batch(max_seq_len, batch_size, feature_len):
    all_data = load_dataset('data/train_sample/', parts=['hits','truth'])
    for data in all_data:
        hit_df, truth_df = data[1], data[2]
        yield from get_data(max_seq_len, batch_size, feature_len, truth_df, hit_df)
        
def get_lv_id(x, y, z):
    row = hits.loc[(hits['x'] == x) & (hits['y'] == y) & (hits['z'] == z)]
    return (int(row.iloc[0][4]), int(row.iloc[0][5]))

In [ ]:
itr = next_batch(19,1,5)
x, labels, _, __ = next(itr)

In [ ]:
x.shape

In [ ]:
labels.shape

#make a new tensor of the same dimension as x, but instead of x,y,z, each point is volume/layer ID
#add lookup code in generator
#FCL for each volume/layer ID pair
#input: 3D vector
#output: 

unique_lv_pairs = []
for row in hits.iloc[:,4:6].itertuples():
    lv_pair = (row[1], row[2])
    if lv_pair not in unique_lv_pairs:
        unique_lv_pairs.append(lv_pair)

fcl_dict = {}
for lv_pair in unique_lv_pairs:
    lv = str(lv_pair[0]) + "-" + str(lv_pair[1])
    inputs = tf.placeholder(tf.float32, [None, 1, 3])
    num_outputs = 30
    layer = tf.contrib.layers.fully_connected(inputs, num_outputs, activation_fn=tf.nn.relu)
    fcl_dict[lv] = layer

def get_fcl(layer_id, volume_id):
    lv_pair = str(layer_id) + "-" + str(volume_id)
    for key, value in fcl_dict.items():
        if lv_pair == key:
            return value

In [3]:
n_steps = 7
n_input_features = 5
n_neurons = 200
n_outputs = 3
learning_rate = 0.001

batch_size = 10

X = tf.placeholder(tf.float32, [None, n_steps, n_input_features], name='input' )
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs], name='label')
    
#uses tanh for inner activations
lstm = tf.contrib.rnn.LSTMCell(num_units = n_neurons, use_peepholes = True)
#lstm = tf.contrib.rnn.LayerNormBasicLSTMCell(num_units = n_neurons)


lstm_cell = tf.contrib.rnn.OutputProjectionWrapper(lstm, output_size = n_outputs)
rnn_outputs, states = tf.nn.dynamic_rnn(lstm_cell, X, dtype = tf.float32)


global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

print_op = tf.Print(rnn_outputs,[rnn_outputs], name='print_pred')
loss_diff = tf.reduce_sum(tf.square(rnn_outputs-y), 2)
mask = tf.sign(tf.reduce_max(tf.abs(y),2))
mask_loss = mask*loss_diff
batch_loss = tf.reduce_mean(mask_loss)


opt = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = opt.minimize(batch_loss, global_step=global_step)

init = tf.global_variables_initializer()
data_itr = next_batch(n_steps, batch_size,n_input_features)


Instructions for updating:
Use the retry module or similar alternatives.


In [ ]:
n_iterations = 10000
batch_size = 50

#saver = tf.train.Saver()
#tf.reset_default_graph()


with tf.name_scope('summaries'):
        tf.summary.scalar('loss', batch_loss)
        tf.summary.histogram('histogram_loss', batch_loss)
        summary_op = tf.summary.merge_all()
        
with tf.Session() as lstm2sess:
   
    #saver.restore(sess, "./lstm_weight1sckpt")
    init.run()
    initial_step = global_step.eval()
    saver = tf.train.Saver()
    writer = tf.summary.FileWriter('./logs/lstm2', lstm2sess.graph)
    ckpt = tf.train.get_checkpoint_state(os.path.dirname('./checkpoints/lstm2/'))
    # if that checkpoint exists, restore from checkpoint
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(lstm2sess, ckpt.model_checkpoint_path)
    
    for iteration in range(n_iterations):
        x_batch, y_batch, _, __ = next(data_itr)
        #print(x_batch.shape)
        #print(y_batch.shape)
        
        
        
        
        ___, loss_batch, summary, test_print = lstm2sess.run([train_op, batch_loss, summary_op, print_op], feed_dict={X:x_batch, y:y_batch})
        writer.add_summary(summary, global_step=iteration)
        
        if iteration %100 ==0:
            mse = batch_loss.eval(feed_dict={X:x_batch, y:y_batch})
            print(iteration, "\tMSE:", mse)
            saver.save(lstm2sess, './checkpoints/lstm2', iteration)
    #saver.save(sess,"./lstm_weights1.ckpt")

0 	MSE: 340311.94
100 	MSE: 353767.94
200 	MSE: 237066.17
300 	MSE: 372085.53
400 	MSE: 432045.3
500 	MSE: 315972.4
600 	MSE: 314309.72
700 	MSE: 333183.5
800 	MSE: 266048.5
900 	MSE: 250634.48
1000 	MSE: 120459.21
1100 	MSE: 188162.92
1200 	MSE: 251840.72
1300 	MSE: 248650.66
1400 	MSE: 150028.42
1500 	MSE: 151188.52
1600 	MSE: 189437.48
1700 	MSE: 193739.72
1800 	MSE: 261068.45
1900 	MSE: 75797.96
2000 	MSE: 128856.26
2100 	MSE: 157265.61


In [ ]:
%tensorboard --logdir=./logs